<a href="https://colab.research.google.com/github/skyshine460/GHG_Direct_Emissions/blob/main/CNN_baseline.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [104]:
import warnings
warnings.filterwarnings("ignore")

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [105]:
# 予測モデルを訓練するためのデータセット
train_df = pd.read_csv('/content/drive/MyDrive/GHGの排出量の予測にチャレンジしよう！（SMBC Group GREEN×DATA Challenge 2024）/提供データ/train.csv')

# 予測モデルに推論（予測)させるデータセット
test_df = pd.read_csv('/content/drive/MyDrive/GHGの排出量の予測にチャレンジしよう！（SMBC Group GREEN×DATA Challenge 2024）/提供データ/test.csv')

In [106]:
# pip install tensorflow

In [107]:
def feature_engineering(df):
    # TRIの差の特徴量
    df['TRI_diff_11-10'] = df['TRI_Air_Emissions_11_in_lbs'] - df['TRI_Air_Emissions_10_in_lbs']
    df['TRI_diff_12-11'] = df['TRI_Air_Emissions_12_in_lbs'] - df['TRI_Air_Emissions_11_in_lbs']
    df['TRI_diff_13-12'] = df['TRI_Air_Emissions_13_in_lbs'] - df['TRI_Air_Emissions_12_in_lbs']

    # TRIの変化率の特徴量
    df['TRI_Growth_Rate_11'] = np.where(
        df['TRI_Air_Emissions_10_in_lbs'].notna() & (df['TRI_Air_Emissions_10_in_lbs'] != 0),
        (df['TRI_Air_Emissions_11_in_lbs'] - df['TRI_Air_Emissions_10_in_lbs']) / df['TRI_Air_Emissions_10_in_lbs'],
        np.nan
    )

    df['TRI_Growth_Rate_12'] = np.where(
        df['TRI_Air_Emissions_11_in_lbs'].notna() & (df['TRI_Air_Emissions_11_in_lbs'] != 0),
        (df['TRI_Air_Emissions_12_in_lbs'] - df['TRI_Air_Emissions_11_in_lbs']) / df['TRI_Air_Emissions_11_in_lbs'],
        np.nan
    )

    df['TRI_Growth_Rate_13'] = np.where(
        df['TRI_Air_Emissions_12_in_lbs'].notna() & (df['TRI_Air_Emissions_12_in_lbs'] != 0),
        (df['TRI_Air_Emissions_13_in_lbs'] - df['TRI_Air_Emissions_12_in_lbs']) / df['TRI_Air_Emissions_12_in_lbs'],
        np.nan
    )

feature_engineering(train_df)
feature_engineering(test_df)

In [108]:

# PrimaryNAICSの最初の２文字の分類
naics_types      = {11: 'Agriculture, Forestry, Fishing and Hunting',
                    21: 'Mining, Quarrying, and Oil and Gas Extraction',
                    22: 'Utilities',
                    23: 'Construction',
                    31: 'Manufacturing',
                    32: 'Manufacturing',
                    33: 'Manufacturing',
                    42: 'Wholesale Trade',
                    44: 'Retail Trade',
                    45: 'Retail Trade',
                    48: 'Transportation and Warehousing',
                    49: 'Transportation and Warehousing',
                    51: 'Information',
                    52: 'Finance and Insurance',
                    53: 'Real Estate and Rental and Leasing',
                    54: 'Professional, Scientific, and Technical Services',
                    55: 'Management of Companies and Enterprises',
                    56: 'Administrative and Support and Waste Management and Remediation Services',
                    61: 'Educational Services',
                    62: 'Health Care and Social Assistance',
                    71: 'Arts, Entertainment, and Recreation',
                    72: 'Accommodation and Food Services',
                    81: 'Other Services (except Public Administration)',
                    92: 'Public Administration'}

# PrimaryNAICSを文字数ごとに分割
train_df['PrimaryNAICS_1-2'] = train_df['PrimaryNAICS'].astype(str).str[:2].astype(int)
test_df['PrimaryNAICS_1-2'] = test_df['PrimaryNAICS'].astype(str).str[:2].astype(int)

# PrimaryNAICSを分類してカテゴリ変数へ変換
train_df['NAICS_Type'] = train_df['PrimaryNAICS_1-2'].map(naics_types).astype('category')
test_df['NAICS_Type'] = test_df['PrimaryNAICS_1-2'].map(naics_types).astype('category')

In [109]:
features = [
                        'TRI_Air_Emissions_10_in_lbs', 'TRI_Air_Emissions_11_in_lbs',
                    'TRI_Air_Emissions_12_in_lbs', 'TRI_Air_Emissions_13_in_lbs','PrimaryNAICS',
                    'GHG_Direct_Emissions_10_in_metric_tons', 'GHG_Direct_Emissions_11_in_metric_tons',
                    'GHG_Direct_Emissions_12_in_metric_tons', 'GHG_Direct_Emissions_13_in_metric_tons',
                        'Latitude','Longitude', 'PrimaryNAICS_1-2'
                    #  'TRI_diff_11-10','TRI_diff_12-11','TRI_diff_13-12', 'TRI_Growth_Rate_11','TRI_Growth_Rate_12','TRI_Growth_Rate_13',
                        # 'NAICS_Type'
]

In [114]:
train = train_df[features]
y_train = train_df['GHG_Direct_Emissions_14_in_metric_tons']
test = test_df[features]

train.fillna(train.mean(), inplace=True)  # 平均値で補完
test.fillna(test.mean(), inplace=True)

In [115]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, Flatten, Dense, Dropout
from sklearn.model_selection import KFold
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_log_error, root_mean_squared_error

# データのスケーリング
target = np.log1p(y_train)

scaler_X = MinMaxScaler(feature_range=(0, 1))
scaler_y = MinMaxScaler(feature_range=(0, 1))
train_scaled = scaler_X.fit_transform(train)
target_scaled = scaler_y.fit_transform(target.values.reshape(-1, 1))
test_scaled = scaler_X.transform(test)

# クロスバリデーションの設定
kf = KFold(n_splits=15, shuffle=True, random_state=42)
fold = 1
train_rmsle_list = []
valid_rmsle_list = []

for train_index, valid_index in kf.split(train_scaled):
    X_train, X_valid = train_scaled[train_index], train_scaled[valid_index]
    y_train, y_valid = target_scaled[train_index], target_scaled[valid_index]

    X_train = X_train.reshape((X_train.shape[0], X_train.shape[1], 1))
    X_valid = X_valid.reshape((X_valid.shape[0], X_valid.shape[1], 1))

    # モデルの構築
    model = Sequential([
        Conv1D(filters=8, kernel_size=2, activation='relu', input_shape=(X_train.shape[1], X_train.shape[2])),
        Conv1D(filters=4, kernel_size=2, activation='relu'),
        Flatten(),
        Dense(1, activation='relu')  # 出力層。予測する値が1つの場合
    ])

    # モデルのコンパイル（RMSEを損失関数として使用）
    def root_mean_squared_error(y_true, y_pred):
        return tf.sqrt(tf.reduce_mean(tf.square(y_true - y_pred)))

    model.compile(optimizer='adam', loss='mean_squared_error', metrics=[root_mean_squared_error])

    # モデルの学習
    model.fit(X_train, y_train, epochs=50, batch_size=32, validation_data=(X_valid, y_valid), verbose=0)

    # 訓練データと検証データでのRMSLEの計算
    y_train_pred = scaler_y.inverse_transform(model.predict(X_train))
    y_train_pred = np.expm1(y_train_pred)
    y_valid_pred = scaler_y.inverse_transform(model.predict(X_valid))
    y_valid_pred = np.expm1(y_valid_pred)

    # y_trainとy_validも元のスケールに戻す
    y_train_original = np.expm1(scaler_y.inverse_transform(y_train))
    y_valid_original = np.expm1(scaler_y.inverse_transform(y_valid))

    # RMSLEの計算
    train_rmsle = np.sqrt(mean_squared_log_error(y_train_original, y_train_pred))
    valid_rmsle = np.sqrt(mean_squared_log_error(y_valid_original, y_valid_pred))

    train_rmsle_list.append(train_rmsle)
    valid_rmsle_list.append(valid_rmsle)

    print(f"Fold {fold} - Train RMSLE: {train_rmsle}, Valid RMSLE: {valid_rmsle}")
    print('-' * 30)
    fold += 1

# クロスバリデーション全体の平均RMSLE
mean_train_rmsle = np.mean(train_rmsle_list)
mean_valid_rmsle = np.mean(valid_rmsle_list)
print(f"[CV] tr:{mean_train_rmsle:.5f} va:{mean_valid_rmsle:.5f}")
print('-' * 30)

# testデータで予測
new_data = test_scaled.reshape(test_scaled.shape[0], -1, 1)  # テストデータ全体を取得
predictions = model.predict(new_data)
test_pred = scaler_y.inverse_transform(predictions)
test_pred = np.expm1(test_pred)


136/136 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step 
Fold 1 - Train RMSLE: 0.796533856593037, Valid RMSLE: 0.7114862506481537
136/136 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 
Fold 2 - Train RMSLE: 0.7796629550493605, Valid RMSLE: 0.8129363851193424
136/136 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 
Fold 3 - Train RMSLE: 0.789124649352885, Valid RMSLE: 0.8276240765133411
136/136 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 
Fold 4 - Train RMSLE: 0.8056232344229974, Valid RMSLE: 0.7310497251064186
136/136 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step 
Fold 5 - Train RMSLE: 0.7989237065744559, Valid RMSLE: 0.8074559897707074
136/136 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step 
Fold 6 - Train RMSLE: 0.7994121581351297, Valid RMSLE: 0.8367409181970169
136/136 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 1

In [116]:
submit = pd.read_csv('/content/drive/MyDrive/GHGの排出量の予測にチャレンジしよう！（SMBC Group GREEN×DATA Challenge 2024）/提供データ/sample_submission.csv', header=None)
submit[1] = test_pred
submit.to_csv('submit_ghg36.csv', header=None, index=False)

# 投稿ファイルの中身を確認
submit.head()

,0,1
0,4655,37811.742188
1,4656,232106.218750
2,4657,40313.574219
3,4658,32024.984375
4,4659,40411.773438
